In [ ]:
import requests
from bs4 import BeautifulSoup

def get_dynamic_headlines(url):
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to fetch {url} (Status: {response.status_code})")
        return []

    soup = BeautifulSoup(response.text, "html.parser")
    headline_candidates = []

    # try likely headline tags/classes
    headline_tags = [
        ('h1', None), ('h2', None), ('h3', None),
        ('a', 'headline'), ('a', 'title'), ('span', 'headline'),
        ('div', 'headline'), ('h2', 'headline'), ('h3', 'headline')
    ]

    for tag, class_name in headline_tags:
        if class_name:
            elements = soup.find_all(tag, class_=class_name)
        else:
            elements = soup.find_all(tag)
        for elem in elements:
            text = elem.get_text().strip()
            if text and len(text.split()) > 3:  # filter for plausible headlines
                headline_candidates.append(text)

    # Remove duplicates, keep order
    seen = set()
    headlines = []
    for headline in headline_candidates:
        if headline not in seen:
            headlines.append(headline)
            seen.add(headline)

    return headlines

# Ask user for a URL
url = input("Enter any news page URL: ").strip()
headlines = get_dynamic_headlines(url)

with open("dynamic_headlines.txt", "w", encoding="utf-8") as f:
    for idx, title in enumerate(headlines, 1):
        f.write(f"{idx}. {title}\n")

print(f"Saved {len(headlines)} headlines to dynamic_headlines.txt")
